In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

In [2]:
# Veriyi yükle
data = pd.read_csv('icml_face_data.csv')

In [3]:
data.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [4]:
# Pikselleri ve etiketleri ayır
pixels = data[' pixels'].apply(lambda x: np.fromstring(x, sep=' '))
pixels = np.vstack(pixels.values)
pixels = pixels.reshape(-1, 48, 48, 1)
pixels = pixels.astype('float32') / 255


In [5]:
labels = to_categorical(data['emotion'])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(pixels, labels, test_size=0.2, random_state=42)


In [7]:
# CNN modelini oluştur
model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))  
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.5))  
model.add(Flatten())
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(7, activation='softmax'))

optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08)

In [8]:

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Erken durdurma ve öğrenme hızı azaltma geri çağrılarını oluşturma
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)



In [9]:

# ModelCheckpoint callback'ini oluşturmak
checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True,  
                             mode='auto') 


In [10]:
# Modeli eğit
model.fit(X_train, y_train, epochs=30, batch_size=86, 
          validation_data=(X_test, y_test),
          callbacks=[early_stopping, learning_rate_reduction, checkpoint])

Epoch 1/30
 83/334 [======>.......................] - ETA: 49s - loss: 1.8751 - accuracy: 0.2286

KeyboardInterrupt: 

In [ ]:
# Modeli kaydet
model.save('my_model.h5')  

AttributeError: module 'keras.preprocessing.image' has no attribute 'load_img'